# Linguistic Analysis: Adjectives Correlation

In [ ]:
import pandas as pd
import os
import re
import numpy as np
import pandas as pd
import pickle
from collections import Counter, defaultdict
import math
import json
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from main_analyze import label_vocab
from main_analyze import get_main_data
df_all = get_main_data(assign_net_income_rank_flag=True)
df_all = df_all.query(" year<=2010 ") 
df_all = df_all.dropna(subset=['abstract_Marketing']).copy()
from main_analyze import label_vocab
interested_label_vocab = label_vocab[:6] 

In [ ]:
from statsmodels.stats import weightstats as stests
import statsmodels.api as sm
year_stratify_scheme = [
    (1980, 2020), 
]
resutls_counter = Counter()
word_to_freq = dict()
word_to_coef = dict()
for year_start, year_end in year_stratify_scheme:
    df_year = df_all.query(" `Disclosure Date`>='{}-01-01'  and `Disclosure Date`<'{}-01-01' ".format(year_start, year_end))
    print(year_start, '-', year_end, 'Number of dockets', len(df_year)  )
    for adj_rank, word in enumerate(adj_counter): 
        occurrence_array = df_year['abstract_Marketing'].str.contains(' ' + word)
        if occurrence_array.sum() < 10:
            resutls_counter['Too Few'] += 1
            continue
        df_tmp = df_year.copy()
        df_tmp['word_occur'] = occurrence_array
        X = df_tmp[['year', 'word_occur'] + interested_label_vocab] 
        Y = df_tmp['income_rank'] 
        X = sm.add_constant(X) 
        model = sm.OLS(Y, X.astype(float)).fit()
        pvalue  = model.pvalues['word_occur'] 
        coef    = model.params['word_occur'] 
        x1 = df_year[occurrence_array]['income_rank']
        x2 = df_year[~occurrence_array]['income_rank']

        if pvalue < 0.05:
            print('{}'.format(adj_rank), end=' ')
            if x1.mean() < x2.mean(): 
                print('[hurts]', end=' ')
                resutls_counter['Hurts'] += 1
            else:
                print('       ', end=' ')
                resutls_counter['Helps'] += 1
            print(word, '\t p-value {:.2e} coef: {:.2e} contain {:.2f}; not contain {:.2f} '.format(pvalue, coef, x1.mean(), x2.mean(), ), 'occurrence', occurrence_array.sum() )

            word_to_freq[word] = np.log(occurrence_array.sum())
            word_to_coef[word] = coef

print('resutls_counter', resutls_counter)

In [ ]:
from wordcloud import WordCloud
coef_max_abs = max([ np.abs(x) for x in word_to_coef.values()])
for del_w in ['plasmid', 'pulse', 'peptide', 'wound', 'therapeutic', 'live']:
    if del_w in word_to_freq:
        del word_to_freq[del_w]
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
sns.set(font_scale=1.4)  
my_colormap = plt.cm.get_cmap('RdBu')

def my_color_func(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    coef = word_to_coef[word]
    r, g, b, _ = np.maximum(0, 255 * np.array( my_colormap(
            coef/ (0.35 * coef_max_abs) + 0.5
            )))
    return "rgb({:.0f}, {:.0f}, {:.0f})".format(r, g, b)

wordcloud = WordCloud(
    scale = 10, 
    background_color="white",
    color_func = my_color_func, 
    ) 
wordcloud.generate_from_frequencies(word_to_freq)
plt.figure( figsize=(8,4), ) 
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")

cbar = plt.colorbar(matplotlib.cm.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=-1, vmax=1), cmap=my_colormap), 
                    fraction=0.046, pad=0.04, ticks=[-1, 1], orientation='horizontal')
cbar.ax.set_xticklabels(['negative correlation', 'positive correlation'])  # vertically oriented colorbar
plt.show()
print('Hype word correlation with net income')